In [15]:
import random
import threading
import time
import numpy as np
import matplotlib.pyplot as plt

class Cell:
    def __init__(self):
        self.is_mine = False
        self.is_revealed = False
        self.adjacent_mines = 0

    def __str__(self):
        if self.is_revealed:
            if self.is_mine:
                return "💣"
            return str(self.adjacent_mines) if self.adjacent_mines > 0 else " "
        return "■"

class Minesweeper:
    def __init__(self, width, height, num_mines):
        self.width = width
        self.height = height
        self.num_mines = num_mines
        self.board = [[Cell() for _ in range(width)] for _ in range(height)]  # 原始踩地雷棋盤
        self.game_over = False
        self.revealed_cells = 0
        self.total_cells = width * height - num_mines
        self.revealed_positions = set()  # 已翻開格子的集合
        self.neighbor_unrevealed_positions = set()  # 未翻開且在數字旁的格子集合
        self.flag_positions = set()  # 插旗子的格子集合
        self.unrevealed_positions = set(i for i in range(height * width))  # 未翻開的格子集合
        self.mine_positions = set()  # 儲存地雷位置的一維索引集合
        self.play_simulation = [[-2 for _ in range(width)] for _ in range(height)]  # 模擬遊玩過程的二維陣列
        self.first_move = True  # 新增旗標以追蹤是否為第一次揭開
        self._reset()

    def _place_mines(self, exclude_x, exclude_y):
        """
        隨機放置地雷，確保 exclude_x, exclude_y 及其鄰近格子不包含地雷。
        """
        exclude_positions = set()
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = exclude_x + dx, exclude_y + dy
                if 0 <= nx < self.height and 0 <= ny < self.width:
                    exclude_positions.add(nx * self.width + ny)

        available_positions = list(set(range(self.width * self.height)) - exclude_positions)
        mine_positions = random.sample(available_positions, self.num_mines)
        self.mine_positions = set(mine_positions)
        for pos in mine_positions:
            x, y = divmod(pos, self.width)
            self.board[x][y].is_mine = True

    def _calculate_adjacent_mines(self):
        for x in range(self.height):
            for y in range(self.width):
                if not self.board[x][y].is_mine:
                    self.board[x][y].adjacent_mines = self._count_adjacent_mines(x, y)

    def _count_adjacent_mines(self, x, y):
        count = 0
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.height and 0 <= ny < self.width and self.board[nx][ny].is_mine:
                    count += 1
        return count

    def _count_playing_adjacent_unrevealed(self, x, y):
        count = 0
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.height and 0 <= ny < self.width and (self.play_simulation[nx][ny] == -2 or self.play_simulation[nx][ny] == -3):
                    count += 1
        return count

    def _count_playing_adjacent_flags(self, x, y):
        count = 0
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.height and 0 <= ny < self.width and self.play_simulation[nx][ny] == -3:
                    count += 1
        return count

    def _update_neighbor_unrevealed_positions(self):
        """
        更新 neighbor_unrevealed_positions 集合，確保集合只包含「未翻開且在數字旁的格子」。
        使用 play_simulation 的數字作為依據，並排除已插旗的格子。
        """
        self.neighbor_unrevealed_positions.clear()  # 重新初始化集合
        for x in range(self.height):
            for y in range(self.width):
                pos_index = x * self.width + y
                # 根據 play_simulation 判斷：未翻開、未插旗且周圍有數字的格子
                if self.play_simulation[x][y] > 0:
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            nx, ny = x + dx, y + dy
                            if (0 <= nx < self.height and
                                0 <= ny < self.width and
                                self.play_simulation[nx][ny] == -2 and
                                (nx * self.width + ny) not in self.flag_positions):
                                self.neighbor_unrevealed_positions.add(nx * self.width + ny)

    def _lowest_property(self, x, y):  # 計算可能為地雷的機率
        sum_prop = 0
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.height and 0 <= ny < self.width and self.play_simulation[nx][ny] > 0:
                    sum_prop += self.play_simulation[nx][ny]
        return sum_prop

    def _reset(self):
        """
        重置遊戲並返回初始狀態（未翻開的棋盤）。
        """
        self.game_over = False
        self.revealed_cells = 0
        self.revealed_positions.clear()
        self.unrevealed_positions = set(i for i in range(self.height * self.width))
        self.neighbor_unrevealed_positions.clear()
        self.flag_positions.clear()
        self.mine_positions.clear()
        self.board = [[Cell() for _ in range(self.width)] for _ in range(self.height)]
        self.play_simulation = [[-2 for _ in range(self.width)] for _ in range(self.height)]  # 重置模擬遊玩過程的二維陣列
        self.first_move = True  # 重置第一次揭開旗標

        # 不放置地雷，等待第一次揭開

        # 返回初始狀態（棋盤格式，形狀為 height x width）
        return np.full((self.height, self.width, 1), -2)

    def flag_cell(self, x, y):
        """
        在指定格子上插旗，並更新相關狀態與集合。
        """
        # 如果格子已翻開或遊戲已結束，直接返回
        if self.board[x][y].is_revealed or self.game_over:
            return False  # 返回是否插旗成功

        pos = x * self.width + y

        # 插旗邏輯
        if pos in self.flag_positions:
            self.flag_positions.remove(pos)
            self.play_simulation[x][y] = -2  # 移除旗標
        else:
            self.flag_positions.add(pos)
            self.play_simulation[x][y] = -3  # 標記為旗標

        # 插旗後更新 neighbor_unrevealed_positions
        self._update_neighbor_unrevealed_positions()

        return pos in self.flag_positions

    def reveal_cell(self, x, y):
        """
        翻開指定的格子，處理擴散邏輯，並從未翻開集合中移除，同時維護集合狀態。
        """
        if self.game_over or self.board[x][y].is_revealed:
            return

        # 如果是第一次揭開，放置地雷，確保第一步安全
        if self.first_move:
            self._place_mines(exclude_x=x, exclude_y=y)
            self._calculate_adjacent_mines()
            self.first_move = False

        # 如果踩到地雷，遊戲結束
        if self.board[x][y].is_mine:
            self.play_simulation[x][y] = -1  # 更新模擬遊玩過程的二維陣列
            self.board[x][y].is_revealed = True
            self.game_over = True
            return

        # 翻開的格子隊列，用於處理擴散
        cells_to_reveal = [(x, y)]

        while cells_to_reveal:
            cx, cy = cells_to_reveal.pop()

            # 如果格子已翻開，跳過
            if self.board[cx][cy].is_revealed:
                continue

            # 翻開當前格子
            self.board[cx][cy].is_revealed = True
            self.revealed_cells += 1
            self.revealed_positions.add(cx * self.width + cy)
            self.unrevealed_positions.discard(cx * self.width + cy)

            # 更新模擬遊玩過程的二維陣列
            self.play_simulation[cx][cy] = self.board[cx][cy].adjacent_mines

            # 如果是空白格子，加入相鄰格子進一步擴散
            if self.board[cx][cy].adjacent_mines == 0:
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx, ny = cx + dx, cy + dy
                        if 0 <= nx < self.height and 0 <= ny < self.width and not self.board[nx][ny].is_revealed:
                            cells_to_reveal.append((nx, ny))

        # 如果所有非地雷格子已翻開，標記為成功
        if self.revealed_cells == self.total_cells and not self.game_over:
            self.game_over = True

        # 統一更新 neighbor_unrevealed_positions
        self._update_neighbor_unrevealed_positions()

    def initialize_with_map(self, map_layout):
        """
        使用自定義地圖初始化遊戲。

        :param map_layout: 二維列表，表示地圖布局。
                           0 表示無地雷，1 表示有地雷。
        """
        self.height = len(map_layout)
        self.width = len(map_layout[0])
        self.num_mines = sum(row.count(1) for row in map_layout)

        # 初始化地圖
        self.board = [[Cell() for _ in range(self.width)] for _ in range(self.height)]
        self.mine_positions.clear()
        self.unrevealed_positions = set(i for i in range(self.height * self.width))

        for x in range(self.height):
            for y in range(self.width):
                if map_layout[x][y] == 1:
                    self.board[x][y].is_mine = True
                    self.mine_positions.add(x * self.width + y)

        # 計算每個格子的鄰近地雷數
        self._calculate_adjacent_mines()

        # 重置其他遊戲狀態
        self.play_simulation = [[-2 for _ in range(self.width)] for _ in range(self.height)]
        self.revealed_cells = 0
        self.revealed_positions.clear()
        self.neighbor_unrevealed_positions.clear()
        self.flag_positions.clear()
        self.game_over = False
        self.total_cells = self.width * self.height - self.num_mines

    def algorithm_reveal_cell(self, callback=None, delay=0.1, Print_board=0):
        """
        利用演算法踩地雷

        :param callback: 用於更新 UI 的回調函數。
        :param delay: 每一步之間的延遲時間（秒）。
        """
        if callback:
            callback()

        while not self.game_over:
            oper_times = 0
            oper_times += self._algo_flag_cell(delay=delay)   # 利用簡單邏輯去插旗子
            oper_times += self._algo_reveal_cell(delay=delay) # 利用簡單邏輯去翻格子

            if oper_times == 0:
                # 啟用121與1221的高級邏輯
                oper_times += self._upscale_logic_reveal_cell(delay=delay)

            if oper_times == 0:
                # 啟用高級邏輯
                oper_times += self._upscale_logic_reveal_cell2(delay=delay)

            if oper_times == 0:
                self. _rand_reveal_cell(delay=delay) #隨機翻

            if callback:
                callback()

            time.sleep(delay)

    def _algo_flag_cell(self, Print_board=0, delay=0.1):
        """
        封裝插旗子的邏輯。
        """
        oper_times = 0
        for x in range(self.height):
            for y in range(self.width):
                if self.play_simulation[x][y] > 0 and self.play_simulation[x][y] == self._count_playing_adjacent_unrevealed(x, y):
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            if dx == 0 and dy == 0:
                                continue
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < self.height and 0 <= ny < self.width and self.play_simulation[nx][ny] == -2:
                                if self.flag_cell(nx, ny):
                                    oper_times += 1
                                    if Print_board >= 3:
                                        print(f"插旗 X:{nx}, Y:{ny}")
        return oper_times

    def _algo_reveal_cell(self, Print_board=0, delay=0.1):
        """
        封裝翻開格子的邏輯。
        """
        oper_times = 0
        for x in range(self.height):
            for y in range(self.width):
                if self.play_simulation[x][y] > 0 and self.play_simulation[x][y] == self._count_playing_adjacent_flags(x, y):
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            if dx == 0 and dy == 0:
                                continue
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < self.height and 0 <= ny < self.width and self.play_simulation[nx][ny] == -2:
                                self.reveal_cell(nx, ny)
                                oper_times += 1
                                if Print_board >= 3:
                                    print(f"翻開 X:{nx}, Y:{ny}")
        return oper_times

    def _rand_reveal_cell(self, Print_board=0, delay=0.1):
        """
        隨機選擇一個未翻開且無插旗的格子，並翻開它。

        :param Print_board: 控制是否打印棋盤，值越大打印越詳細。
        :param delay: 每一步之間的延遲時間（秒）。
        :return: 操作次數（1 表示成功翻開一個格子，0 表示沒有可翻開的格子）。
        """
        # 獲取所有未翻開且未插旗的格子
        possible_positions = list(self.unrevealed_positions - self.flag_positions)
        
        # 如果沒有符合條件的格子，返回 0
        if not possible_positions:
            return 0
        
        # 隨機選擇一個格子
        random_element = random.choice(possible_positions)
        X, Y = divmod(random_element, self.width)
        
        # 翻開選中的格子
        self.reveal_cell(X, Y)
        
        # 如果需要打印棋盤，顯示翻開的格子位置
        if Print_board >= 3:
            print(f"隨機翻開 X:{X}, Y:{Y}")
        
        # 返回操作次數
        return 1

    def _upscale_logic_reveal_cell(self, Print_board=0, delay=0.1):
        temp_play_simulation = [row[:] for row in self.play_simulation]  # 模擬遊玩過程的二維陣列 (扣掉周圍旗子的數字)
        oper_times = 0
        for x in range(self.height):
            for y in range(self.width):
                if self.play_simulation[x][y] >= 0:
                    temp_play_simulation[x][y] -= self._count_playing_adjacent_flags(x, y)

        # 直的121
        for x in range(self.height - 2):
            for y in range(self.width):
                if (temp_play_simulation[x][y] == 1 and
                    temp_play_simulation[x + 1][y] == 2 and
                    temp_play_simulation[x + 2][y] == 1):
                    if (0 <= x + 1 < self.height and
                        0 <= y - 1 < self.width and
                        self.play_simulation[x + 1][y - 1] == -2):
                        self.reveal_cell(x + 1, y - 1)
                        if Print_board >= 3:
                            print(f"利用121邏輯翻開 X:{x + 1}, Y:{y - 1}")
                        oper_times += 1
                    if (0 <= x + 1 < self.height and
                        0 <= y + 1 < self.width and
                        self.play_simulation[x + 1][y + 1] == -2):
                        self.reveal_cell(x + 1, y + 1)
                        if Print_board >= 3:
                            print(f"利用121邏輯翻開 X:{x + 1}, Y:{y + 1}")
                        oper_times += 1

        # 橫的121
        for x in range(self.height):
            for y in range(self.width - 2):
                if (temp_play_simulation[x][y] == 1 and
                    temp_play_simulation[x][y + 1] == 2 and
                    temp_play_simulation[x][y + 2] == 1):
                    if (0 <= x - 1 < self.height and
                        0 <= y + 1 < self.width and
                        self.play_simulation[x - 1][y + 1] == -2):
                        self.reveal_cell(x - 1, y + 1)
                        if Print_board >= 3:
                            print(f"利用121邏輯翻開 X:{x - 1}, Y:{y + 1}")
                        oper_times += 1
                    if (0 <= x + 1 < self.height and
                        0 <= y + 1 < self.width and
                        self.play_simulation[x + 1][y + 1] == -2):
                        self.reveal_cell(x + 1, y + 1)
                        if Print_board >= 3:
                            print(f"利用121邏輯翻開 X:{x + 1}, Y:{y + 1}")
                        oper_times += 1

        # 直的1221
        for x in range(self.height - 3):
            for y in range(self.width):
                if (temp_play_simulation[x][y] == 1 and
                    temp_play_simulation[x + 1][y] == 2 and
                    temp_play_simulation[x + 2][y] == 2 and
                    temp_play_simulation[x + 3][y] == 1):
                    if (0 <= x < self.height and
                        0 <= y - 1 < self.width and
                        self.play_simulation[x][y - 1] == -2):
                        self.reveal_cell(x, y - 1)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x}, Y:{y - 1}")
                        oper_times += 1
                    if (0 <= x < self.height and
                        0 <= y + 1 < self.width and
                        self.play_simulation[x][y + 1] == -2):
                        self.reveal_cell(x, y + 1)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x}, Y:{y + 1}")
                        oper_times += 1
                    if (0 <= x + 3 < self.height and
                        0 <= y - 1 < self.width and
                        self.play_simulation[x + 3][y - 1] == -2):
                        self.reveal_cell(x + 3, y - 1)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x + 3}, Y:{y - 1}")
                        oper_times += 1
                    if (0 <= x + 3 < self.height and
                        0 <= y + 1 < self.width and
                        self.play_simulation[x + 3][y + 1] == -2):
                        self.reveal_cell(x + 3, y + 1)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x + 3}, Y:{y + 1}")
                        oper_times += 1

        # 橫的1221
        for x in range(self.height):
            for y in range(self.width - 3):
                if (temp_play_simulation[x][y] == 1 and
                    temp_play_simulation[x][y + 1] == 2 and
                    temp_play_simulation[x][y + 2] == 2 and
                    temp_play_simulation[x][y + 3] == 1):
                    if (0 <= x - 1 < self.height and
                        0 <= y < self.width and
                        self.play_simulation[x - 1][y] == -2):
                        self.reveal_cell(x - 1, y)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x - 1}, Y:{y}")
                        oper_times += 1
                    if (0 <= x + 1 < self.height and
                        0 <= y < self.width and
                        self.play_simulation[x + 1][y] == -2):
                        self.reveal_cell(x + 1, y)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x + 1}, Y:{y}")
                        oper_times += 1
                    if (0 <= x - 1 < self.height and
                        0 <= y + 3 < self.width and
                        self.play_simulation[x - 1][y + 3] == -2):
                        self.reveal_cell(x - 1, y + 3)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x - 1}, Y:{y + 3}")
                        oper_times += 1
                    if (0 <= x + 1 < self.height and
                        0 <= y + 3 < self.width and
                        self.play_simulation[x + 1][y + 3] == -2):
                        self.reveal_cell(x + 1, y + 3)
                        if Print_board >= 3:
                            print(f"利用1221邏輯翻開 X:{x + 1}, Y:{y + 3}")
                        oper_times += 1

        return oper_times

    def _upscale_logic_reveal_cell2(self, Print_board=0, delay=0.1):
        temp_play_simulation = [row[:] for row in self.play_simulation]  # 模擬遊玩過程的二維陣列 (扣掉周圍旗子的數字)
        oper_times = 0
        for x in range(self.height):
            for y in range(self.width):
                if self.play_simulation[x][y] >= 0:
                    temp_play_simulation[x][y] -= self._count_playing_adjacent_flags(x, y)

        for x in range(self.height):
            for y in range(self.width):
                if self.play_simulation[x][y] == -2:
                    continue
                mark = [[0 for _ in range(self.width)] for _ in range(self.height)]
                total_mark = 0
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.height and 0 <= ny < self.width and self.play_simulation[nx][ny] == -2:
                            mark[nx][ny] = 1
                            total_mark += 1

                for dx in [-2, -1, 0, 1, 2]:
                    for dy in [-2, -1, 0, 1, 2]:
                        if dx == 0 and dy == 0:
                            continue
                        nx, ny = x + dx, y + dy
                        if (0 <= nx < self.height and
                            0 <= ny < self.width and
                            self.play_simulation[nx][ny] > 0 and
                            temp_play_simulation[x][y] == temp_play_simulation[nx][ny]):
                            now_mark = 0
                            for Dx in [-1, 0, 1]:
                                for Dy in [-1, 0, 1]:
                                    Nx, Ny = nx + Dx, ny + Dy
                                    if (0 <= Nx < self.height and
                                        0 <= Ny < self.width and
                                        mark[Nx][Ny] == 1):
                                        now_mark += 1
                            if total_mark == now_mark:
                                for Dx in [-1, 0, 1]:
                                    for Dy in [-1, 0, 1]:
                                        Nx, Ny = nx + Dx, ny + Dy
                                        if (0 <= Nx < self.height and
                                            0 <= Ny < self.width and
                                            mark[Nx][Ny] == 0 and
                                            self.play_simulation[Nx][Ny] == -2):
                                            self.reveal_cell(Nx, Ny)
                                            if Print_board >= 3:
                                                print(f"利用高級邏輯排除 X:{Nx}, Y:{Ny}")
                                            oper_times += 1
                                            return 1

        return oper_times



In [16]:
import tkinter as tk
from tkinter import messagebox
import threading
import time
import numpy as np
from tensorflow.keras.models import load_model  # 確保已安裝 TensorFlow/Keras

# 假設您已經有 Minesweeper 類別的定義，這裡只展示 MinesweeperUI 的修改部分

class MinesweeperUI:
    def __init__(self, root, width=9, height=9, num_mines=10):
        self.root = root
        self.width = width
        self.height = height
        self.num_mines = num_mines
        self.game = Minesweeper(width, height, num_mines)
        self.buttons = {}
        self.step = 0  # 標記是否是第一次點擊
        self.remaining_mines = self.num_mines
        self.timer_seconds = 0
        self.timer_running = False

        # 計數變數
        self.total_games = 0
        self.total_wins = 0

        # 新增旗標以追蹤 AI 是否為第一步
        self.first_move_ai = True

        # 新增旗標以追蹤是否正在重置遊戲
        self.resetting = False

        # 定義不同難度對應的模型路徑
        self.model_paths = {
            'Beginner': r"D:\Users\misso\Downloads\Minesweeper_Game\Minesweeper_Game\Minesweeper_model\minesweeper_Beginner_50epochs_nbr_cor_up_91win-rate.h5",
            'Intermediate': r"D:\Users\misso\Downloads\Minesweeper_Game\Minesweeper_Game\Minesweeper_model\minesweeper_Intermediate_50epochs_nbr_cor_up_63%win-rate.h5",
            'Expert': r"D:\Users\misso\Downloads\Minesweeper_Game\Minesweeper_Game\Minesweeper_model\minesweeper_Expert_50epochs_nbr_cor_up.h5"
        }

        # 當前選擇的難度
        self.current_difficulty = 'Beginner'

        # 初始化模型為 None
        self.model = None

        self.create_widgets()
        self.update_buttons()

        # 初始載入預設難度的模型
        self.load_model_for_difficulty(self.current_difficulty)

    def load_model_for_difficulty(self, difficulty):
        """
        加載指定難度的模型。
        
        :param difficulty: 字串，表示難度（'Beginner', 'Intermediate', 'Expert'）
        """
        model_path = self.model_paths.get(difficulty)
        if not model_path:
            messagebox.showerror("模型錯誤", f"未找到難度 '{difficulty}' 對應的模型路徑。")
            self.model = None
            return

        try:
            self.model = load_model(model_path)
            print(f"{difficulty} 模型載入成功。")
        except Exception as e:
            messagebox.showerror("模型載入錯誤", f"無法載入模型 '{difficulty}': {e}")
            self.model = None

    def create_widgets(self):
        top_frame = tk.Frame(self.root)
        top_frame.pack(pady=10)

        # 遊戲設置的標籤和輸入欄位
        tk.Label(top_frame, text="寬度:", font=("Helvetica", 12)).grid(row=0, column=0, padx=5)
        self.width_entry = tk.Entry(top_frame, width=5, font=("Helvetica", 12))
        self.width_entry.insert(0, str(self.width))
        self.width_entry.grid(row=0, column=1, padx=5)

        tk.Label(top_frame, text="高度:", font=("Helvetica", 12)).grid(row=0, column=2, padx=5)
        self.height_entry = tk.Entry(top_frame, width=5, font=("Helvetica", 12))
        self.height_entry.insert(0, str(self.height))
        self.height_entry.grid(row=0, column=3, padx=5)

        tk.Label(top_frame, text="地雷數量:", font=("Helvetica", 12)).grid(row=0, column=4, padx=5)
        self.mines_entry = tk.Entry(top_frame, width=5, font=("Helvetica", 12))
        self.mines_entry.insert(0, str(self.num_mines))
        self.mines_entry.grid(row=0, column=5, padx=5)

        # 預設難度按鈕
        preset_frame = tk.Frame(top_frame)
        preset_frame.grid(row=0, column=6, padx=10)

        self.beginner_button = tk.Button(
            preset_frame,
            text="Beginner",
            command=self.set_beginner,
            font=("Helvetica", 10, "bold"),
            width=10
        )
        self.beginner_button.pack(side=tk.LEFT, padx=2)

        self.intermediate_button = tk.Button(
            preset_frame,
            text="Intermediate",
            command=self.set_intermediate,
            font=("Helvetica", 10, "bold"),
            width=12
        )
        self.intermediate_button.pack(side=tk.LEFT, padx=2)

        self.expert_button = tk.Button(
            preset_frame,
            text="Expert",
            command=self.set_expert,
            font=("Helvetica", 10, "bold"),
            width=10
        )
        self.expert_button.pack(side=tk.LEFT, padx=2)

        # Run Simulation 按鈕
        self.run_sim_button = tk.Button(
            preset_frame,
            text="交給演算法踩",
            command=self.run_simulation,
            font=("Helvetica", 10, "bold"),
            width=15
        )
        self.run_sim_button.pack(side=tk.LEFT, padx=2)

        # 新增 AI 按鈕
        self.ai_button = tk.Button(
            preset_frame,
            text="交給AI踩",
            command=self.run_ai,
            font=("Helvetica", 10, "bold"),
            width=10
        )
        self.ai_button.pack(side=tk.LEFT, padx=2)

        # 重置遊戲按鈕
        self.reset_button = tk.Button(
            top_frame,
            text="重置遊戲",
            command=self.reset_game,
            font=("Helvetica", 14, "bold"),
            width=10
        )
        self.reset_button.grid(row=0, column=7, padx=10)

        # 計時器標籤
        self.timer_label = tk.Label(self.root, text="時間: 0", font=("Helvetica", 14, "bold"))
        self.timer_label.pack(pady=5)

        # 剩餘地雷標籤
        self.mines_label = tk.Label(self.root, text=f"剩餘地雷: {self.remaining_mines}", font=("Helvetica", 14, "bold"))
        self.mines_label.pack(pady=5)
        
        # 棋盤框架
        self.board_frame = tk.Frame(self.root)
        self.board_frame.pack()

        self.create_board()

    def create_board(self):
        # 創建棋盤按鈕
        for x in range(self.height):
            for y in range(self.width):
                btn = tk.Button(
                    self.board_frame,
                    width=2,
                    height=1,
                    font=("Helvetica", 10, "bold"),
                    command=lambda x=x, y=y: self.on_left_click(x, y)
                )
                btn.bind("<Button-3>", lambda event, x=x, y=y: self.on_right_click(x, y))
                btn.grid(row=x, column=y)
                self.buttons[(x, y)] = btn

    def reset_game(self):
        # 停止計時器並重設時間
        self.stop_timer()
        self.timer_seconds = 0
        self.timer_label.config(text="時間: 0")
        
        # 讀取用戶輸入的設置
        try:
            new_width = int(self.width_entry.get())
            new_height = int(self.height_entry.get())
            new_num_mines = int(self.mines_entry.get())
            
            if new_width <= 0 or new_height <= 0:
                raise ValueError("寬度和高度必須是正整數。")
            if new_num_mines < 0 or new_num_mines >= new_width * new_height:
                raise ValueError("地雷數量必須在合理範圍內。")

        except ValueError as ve:
            messagebox.showerror("輸入錯誤", str(ve))
            return

        # 更新遊戲設置
        self.width = new_width
        self.height = new_height
        self.num_mines = new_num_mines
        self.remaining_mines = new_num_mines
        self.mines_label.config(text=f"剩餘地雷: {self.remaining_mines}")

        # 重新初始化遊戲
        self.game = Minesweeper(self.width, self.height, self.num_mines)
        self.step = 0

        # 重置 AI 的第一步旗標
        self.first_move_ai = True

        # 銷毀現有的按鈕
        for btn in self.buttons.values():
            btn.destroy()
        self.buttons.clear()

        # 重建棋盤框架
        self.board_frame.destroy()
        self.board_frame = tk.Frame(self.root)
        self.board_frame.pack()
        self.create_board()

        # 重置正在重置旗標
        self.resetting = False

        # 根據當前難度加載相應模型
        self.load_model_for_difficulty(self.current_difficulty)

        self.update_buttons()

    def set_beginner(self):
        self.width_entry.delete(0, tk.END)
        self.width_entry.insert(0, "9")
        self.height_entry.delete(0, tk.END)
        self.height_entry.insert(0, "9")
        self.mines_entry.delete(0, tk.END)
        self.mines_entry.insert(0, "10")
        self.current_difficulty = 'Beginner'  # 設置當前難度
        self.reset_game()

    def set_intermediate(self):
        self.width_entry.delete(0, tk.END)
        self.width_entry.insert(0, "16")
        self.height_entry.delete(0, tk.END)
        self.height_entry.insert(0, "16")
        self.mines_entry.delete(0, tk.END)
        self.mines_entry.insert(0, "40")
        self.current_difficulty = 'Intermediate'  # 設置當前難度
        self.reset_game()

    def set_expert(self):
        self.width_entry.delete(0, tk.END)
        self.width_entry.insert(0, "30")
        self.height_entry.delete(0, tk.END)
        self.height_entry.insert(0, "16")
        self.mines_entry.delete(0, tk.END)
        self.mines_entry.insert(0, "99")
        self.current_difficulty = 'Expert'  # 設置當前難度
        self.reset_game()

    def on_left_click(self, x, y):
        if self.game.game_over:
            return

        if not self.timer_running:
            self.start_timer()

        pos = x * self.width + y

        # 如果格子已經被插旗，左鍵點擊不執行任何操作
        if pos in self.game.flag_positions:
            return

        # 第一次點擊的邏輯，保證點擊的格子不是地雷
        if self.step == 0:
            if self.game.board[x][y].is_mine or self.game.board[x][y].adjacent_mines != 0:
                # 重置遊戲，重新開始
                self.game._reset()
                self.game.reveal_cell(x, y)
            else:
                self.game.reveal_cell(x, y)
            self.update_buttons()
            self.step += 1
            return

        cell = self.game.board[x][y]

        if cell.is_revealed and cell.adjacent_mines > 0:
            flags_around = self.game._count_playing_adjacent_flags(x, y)
            if cell.adjacent_mines == flags_around:
                # 翻開周圍所有未翻開的格子
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < self.height and 0 <= ny < self.width:
                            pos_neighbor = nx * self.width + ny
                            if pos_neighbor not in self.game.flag_positions and not self.game.board[nx][ny].is_revealed:
                                self.game.reveal_cell(nx, ny)
                                self.step += 1
                                self.update_buttons()
        else:
            print("我點了一下左鍵")
            self.game.reveal_cell(x, y)
            if not self.game.board[x][y].is_mine:
                self.step += 1

        self.update_buttons()

        if self.game.game_over:
            self.stop_timer()
            self.total_games += 1
            if self.game.revealed_cells == self.game.total_cells:
                self.total_wins += 1
                result = "勝利"
            else:
                result = "失敗"

            win_rate = (self.total_wins / self.total_games) * 100 if self.total_games > 0 else 0
            print(f"遊玩場次: {self.total_games}, 勝率: {win_rate:.2f}%, 結果: {result}")
            self.show_all_mines()

    def on_right_click(self, x, y):
        if self.game.game_over:
            return

        pos = x * self.width + y

        if self.game.board[x][y].is_revealed:
            return

        flagged = self.game.flag_cell(x, y)
        if flagged:
            self.remaining_mines -= 1
        else:
            self.remaining_mines += 1
        self.mines_label.config(text=f"剩餘地雷: {self.remaining_mines}")
        self.update_buttons()

    def update_buttons(self):
        if self.resetting:
            return  # 當正在重置時，跳過更新

        for x in range(self.height):
            for y in range(self.width):
                cell = self.game.board[x][y]
                btn = self.buttons.get((x, y))
                if not btn:
                    continue
                pos = x * self.width + y

                if pos in self.game.flag_positions:
                    btn.config(text="🚩", bg="#C6C6C6", fg="red", font=("Helvetica", 10, "bold"))
                elif cell.is_revealed:
                    if cell.is_mine:
                        btn.config(text="💣", bg="red", fg="black", font=("Helvetica", 10, "bold"))
                    elif cell.adjacent_mines > 0:
                        btn.config(
                            text=str(cell.adjacent_mines),
                            bg="#FFFFFF",
                            fg=self.get_color(cell.adjacent_mines),
                            font=("Helvetica", 10, "bold")
                        )
                    else:
                        btn.config(text="", bg="#FFFFFF")
                else:
                    btn.config(text="", bg="#C6C6C6")

    def show_all_mines(self):
        if self.resetting:
            return  # 當正在重置時，跳過顯示

        for x in range(self.height):
            for y in range(self.width):
                cell = self.game.board[x][y]
                btn = self.buttons.get((x, y))
                if not btn:
                    continue

                if cell.is_mine:
                    if cell.is_revealed:
                        # 被踩到的地雷格子：顯示地雷字元，背景設為紅色
                        btn.config(text="💣", bg="red", fg="black", font=("Helvetica", 10, "bold"))
                    else:
                        # 未揭開的地雷格子：顯示地雷字元，背景保持未揭開顏色
                        btn.config(text="💣", bg="#C6C6C6", fg="black", font=("Helvetica", 10, "bold"))
        # 保持其他格子不變

    def get_color(self, number):
        colors = {
            1: "blue",
            2: "green",
            3: "red",
            4: "darkblue",
            5: "brown",
            6: "cyan",
            7: "gray",
            8: "black"
        }
        return colors.get(number, "black")

    def run_simulation(self):
        if self.game.game_over:
            return

        self.disable_user_interaction()

        # 重建棋盤框架
        self.board_frame.destroy()
        self.board_frame = tk.Frame(self.root)
        self.board_frame.pack()
        self.create_board()
        time.sleep(0.1)

        # 在新的線程中運行演算法
        simulation_thread = threading.Thread(target=self.game.algorithm_reveal_cell, args=(self.ui_update_callback, 0.1, 3))
        simulation_thread.start()

        # 開始監控遊戲結束
        monitor_thread = threading.Thread(target=self.monitor_game_over)
        monitor_thread.start()

    def monitor_game_over(self):
        while not self.game.game_over:
            time.sleep(0.1)
        self.root.after(0, self.on_game_over)

    def on_game_over(self):
        self.update_buttons()
        self.show_all_mines()
        self.stop_timer()
        self.total_games += 1
        if self.game.revealed_cells == self.game.total_cells:
            self.total_wins += 1
            result = "勝利"
        else:
            result = "失敗"

        win_rate = (self.total_wins / self.total_games) * 100 if self.total_games > 0 else 0
        print(f"遊玩場次: {self.total_games}, 勝率: {win_rate:.2f}%, 結果: {result}")

        self.enable_user_interaction()

    def ui_update_callback(self):
        """
        用於演算法中的回調函數，更新 UI。
        使用 root.after 確保在主線程中執行。
        """
        self.root.after(0, self.update_buttons)

    def disable_user_interaction(self):
        for btn in self.buttons.values():
            btn.config(state=tk.DISABLED)
        self.run_sim_button.config(state=tk.DISABLED)
        self.ai_button.config(state=tk.DISABLED)  # 禁用 AI 按鈕
        self.reset_button.config(state=tk.DISABLED)
        self.beginner_button.config(state=tk.DISABLED)
        self.intermediate_button.config(state=tk.DISABLED)
        self.expert_button.config(state=tk.DISABLED)

    def enable_user_interaction(self):
        for btn in self.buttons.values():
            btn.config(state=tk.NORMAL)
        self.run_sim_button.config(state=tk.NORMAL)
        self.ai_button.config(state=tk.NORMAL)  # 啟用 AI 按鈕
        self.reset_button.config(state=tk.NORMAL)
        self.beginner_button.config(state=tk.NORMAL)
        self.intermediate_button.config(state=tk.NORMAL)
        self.expert_button.config(state=tk.NORMAL)

    def start_timer(self):
        if not self.timer_running:
            self.timer_running = True
            self.timer_update()

    def timer_update(self):
        if self.timer_running:
            self.timer_seconds += 1
            self.timer_label.config(text=f"時間: {self.timer_seconds}")
            self.root.after(1000, self.timer_update)

    def stop_timer(self):
        self.timer_running = False

    # 新增 AI 按鈕的處理方法
    def run_ai(self):
        """
        此方法由 AI 按鈕觸發，啟動 AI 處理線程。
        """
        if self.game.game_over:
            return

        if not self.model:
            messagebox.showerror("模型未載入", "請先載入 AI 模型。")
            return

        self.disable_user_interaction()

        # 在新的線程中運行 AI 處理
        ai_thread = threading.Thread(target=self.ai_process)
        ai_thread.start()

    def ai_process(self):
        """
        AI 處理線程，負責與模型互動並執行遊戲步驟。
        確保第一步不踩到地雷。如果踩到地雷，重新生成地圖並重試。
        不更新遊戲畫面直到第一步操作成功。
        """
        # 重建棋盤框架
        self.board_frame.destroy()
        self.board_frame = tk.Frame(self.root)
        self.board_frame.pack()
        self.create_board()

        while True:
            # 確保遊戲未結束
            if self.game.game_over:
                break
            
            # 獲取當前棋盤狀態
            board_array = np.array(self.game.play_simulation)

            # 將 play_simulation 中的負值映射到 -8 以標識不可操作的格子
            board_array[board_array < 0] = -8

            print("當前棋盤狀態：")
            print(board_array)

            # 將棋盤正規化到 [0, 1] 範圍
            normalized_board = (board_array) / 8.0  # 將所有數值轉換為 0 到 1 之間
            print("正規化後的棋盤：")
            print(normalized_board)

            # 擴展維度以符合模型輸入要求
            input_board = normalized_board.reshape(1, self.height, self.width, 1)

            # 使用模型預測下一步
            prediction = self.model.predict(input_board, verbose=0)
            prediction = prediction.reshape(self.height, self.width)
            print("模型預測結果：")
            print(prediction)

            # 創建掩碼：包括所有已翻開或已插旗的格子
            mask = (board_array != -8)

            # 將掩碼位置的預測值設置為 -inf，避免選擇這些格子
            prediction[mask] = -np.inf
            print("應用掩碼後的預測結果：")
            print(prediction)

            # 獲取置信度最高的格子
            if np.all(prediction == -np.inf):
                # 如果所有可操作的格子都已處理，遊戲結束
                print("所有可操作的格子都已處理，遊戲結束。")
                break

            x, y = np.unravel_index(np.argmax(prediction), prediction.shape)
            print(f"AI 選擇翻開格子: ({x}, {y})")

            # 翻開預測的格子
            self.game.reveal_cell(x, y)

            # 更新 UI
            self.root.after(0, self.update_buttons)

            # 檢查是否為第一步且踩到地雷
            if self.first_move_ai and self.game.board[x][y].is_mine:
                print("第一步踩到地雷，重新生成地圖。")
                self.resetting = True  # 設置正在重置旗標
                # 重新生成地圖，並確保不更新遊戲畫面
                self.root.after(0, self.reset_game)
                # 延遲一點時間以確保遊戲狀態已更新
                time.sleep(0.1)
                self.resetting = False  # 重置旗標
                continue  # 重新嘗試

            # 如果是第一步且未踩到地雷，標記第一步已完成
            if self.first_move_ai:
                self.first_move_ai = False
                self.root.after(0, self.update_buttons)
            else:
                # 更新遊戲畫面
                self.root.after(0, self.update_buttons)

            # 檢查遊戲是否結束
            if self.game.game_over:
                break

            # 延遲 0.1 秒
            time.sleep(0.1)

        # 遊戲結束後處理
        self.root.after(0, self.on_ai_game_over)

    def on_ai_game_over(self):
        self.update_buttons()
        self.show_all_mines()
        self.stop_timer()
        self.total_games += 1
        if self.game.revealed_cells == self.game.total_cells:
            self.total_wins += 1
            result = "勝利"
        else:
            result = "失敗"

        win_rate = (self.total_wins / self.total_games) * 100 if self.total_games > 0 else 0
        print(f"遊玩場次: {self.total_games}, 勝率: {win_rate:.2f}%, 結果: {result}")

        self.enable_user_interaction()


In [17]:
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Minesweeper")

    # 設定遊戲尺寸和地雷數量
    game_width = 9
    game_height = 9
    num_mines = 10

    app = MinesweeperUI(root, width=game_width, height=game_height, num_mines=num_mines)
    root.mainloop()


Beginner 模型載入成功。
遊玩場次: 1, 勝率: 0.00%, 結果: 失敗


Beginner 模型載入成功。
遊玩場次: 2, 勝率: 50.00%, 結果: 勝利


Beginner 模型載入成功。
遊玩場次: 3, 勝率: 66.67%, 結果: 勝利


Beginner 模型載入成功。
當前棋盤狀態：
[[-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]]
正規化後的棋盤：
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]
模型預測結果：
[[0.02441931 0.01001401 0.00648192 0.00192564 0.0023168  0.00511156
  0.00743332 0.01226344 0.01051433]
 [0.00978559 0.01361367 0.002717   0.00100862 0.00146148 0.00273785
  0.00419804 0.00627805 0.01287965]
 [0.00640999 0.00561199 0.00583357 0.00613616 0.00365843 0.00831417
  0.01532011 0.00807195 0.03402535]
 [0.00440658 0.00207171

Beginner 模型載入成功。
當前棋盤狀態：
[[-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]
 [-8 -8 -8 -8 -8 -8 -8 -8 -8]]
正規化後的棋盤：
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]
模型預測結果：
[[0.02441931 0.01001401 0.00648192 0.00192564 0.0023168  0.00511156
  0.00743332 0.01226344 0.01051433]
 [0.00978559 0.01361367 0.002717   0.00100862 0.00146148 0.00273785
  0.00419804 0.00627805 0.01287965]
 [0.00640999 0.00561199 0.00583357 0.00613616 0.00365843 0.00831417
  0.01532011 0.00807195 0.03402535]
 [0.00440658 0.00207171

...

Expert 模型載入成功。
我點了一下左鍵
我點了一下左鍵
遊玩場次: 12, 勝率: 33.33%, 結果: 失敗


Expert 模型載入成功。
遊玩場次: 13, 勝率: 30.77%, 結果: 失敗


Expert 模型載入成功。
遊玩場次: 14, 勝率: 28.57%, 結果: 失敗


Expert 模型載入成功。
遊玩場次: 15, 勝率: 26.67%, 結果: 失敗


Expert 模型載入成功。
遊玩場次: 16, 勝率: 25.00%, 結果: 失敗
